In [ ]:
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os
from catboost import CatBoostClassifier, Pool
import gc
pd.options.display.float_format = '{:,.2f}'.format
import shap
import catboost
#

In [ ]:
# Определяем директорию для сохранения моделей
save_dir = 'saved_models'
# Создаем папку для моделей, если она не существует
os.makedirs(save_dir, exist_ok=True)

# Определяем директорию для сохранения результатов предсказаний
output_folder = "results_folder"
# Создаем папку для результатов, если она не существует
os.makedirs(output_folder, exist_ok=True)

# Определяем пути для сохранения результатов предсказаний
output_file_path_1 = os.path.join(output_folder, "result_1.csv")
output_file_path_2 = os.path.join(output_folder, "result_2.csv")
output_file_path_3 = os.path.join(output_folder, "result_3.csv")

# Определяем директорию, где хранятся разделенные данные
data_dir = "data_split"

# Определяем полные пути к файлам тренировочных и тестовых данных для каждого блока
train_path_1 = os.path.join(data_dir, "train_block_1.parquet")
test_path_1 = os.path.join(data_dir, "test_block_1.parquet")

train_path_2 = os.path.join(data_dir, "train_block_2.parquet")
test_path_2 = os.path.join(data_dir, "test_block_2.parquet")

train_path_3 = os.path.join(data_dir, "train_block_3.parquet")
test_path_3 = os.path.join(data_dir, "test_block_3.parquet")

# Определяем полные пути для сохранения каждой модели CatBoost
catboost_model_path_1 = os.path.join(save_dir, 'catboost_model_1.cbm')
catboost_model_path_2 = os.path.join(save_dir, 'catboost_model_2.cbm')
catboost_model_path_3 = os.path.join(save_dir, 'catboost_model_3.cbm')

In [ ]:
def filter_data_for_training(dataset, test_data, block_name, target_col='target', reduce_frac=0.1):
    """
    Фильтрует тренировочные данные для обучения модели, сохраняя все примеры с target = 1, 
    а также совпадающие примеры по указанной комбинации ключей в зависимости от блока.
    Из оставшихся данных с target = 0 случайным образом выбирает заданную долю.

    Параметры:
    - dataset: DataFrame с тренировочными данными.
    - test_data: DataFrame с тестовыми данными для проверки совпадающих ключей.
    - block_name: Название блока ('train_block_1', 'train_block_2', или 'train_block_3').
    - target_col: Название столбца с целевой переменной.
    - reduce_frac: Доля данных с target = 0, которые нужно оставить.

    Возвращает:
    - DataFrame с отфильтрованными данными.
    """

    # Определяем ключи для каждого блока
    if block_name == 'train_block_1':
        keys = ['user_id', 'logcat_id', 'adv_campaign_id']
    elif block_name == 'train_block_2':
        keys = ['user_id', 'logcat_id']
    elif block_name == 'train_block_3':
        keys = None  # Для train_block_3 не нужны ключи
    else:
        raise ValueError("Некорректное значение block_name. Ожидается 'train_block_1', 'train_block_2' или 'train_block_3'.")

    # Шаг 1: Отбираем все данные с target = 1
    target_one_data = dataset[dataset[target_col] == 1]

    if keys:
        # Шаг 2: Определяем уникальные комбинации ключей в тестовом наборе
        test_keys = test_data[keys].drop_duplicates()
        
        # Шаг 3: Отбираем совпадающие по ключам записи из тренировочных данных
        matching_data = dataset.merge(test_keys, on=keys, how='inner')
        
        # Шаг 4: Отбираем оставшиеся данные с target = 0, которые не входят в matching_data
        target_zero_data = dataset[(dataset[target_col] == 0) & (~dataset.index.isin(matching_data.index))]
        
        # Шаг 5: Случайным образом выбираем reduce_frac от оставшихся данных с target = 0
        target_zero_sampled = target_zero_data.sample(frac=reduce_frac, random_state=42)
        
        # Объединяем данные с target = 1, совпадающие по ключам, и случайную выборку оставшихся данных с target = 0
        filtered_data = pd.concat([target_one_data, matching_data, target_zero_sampled])
    else:
        # Если блок `train_block_3`, то только отбираем target = 1 и случайную выборку
        target_zero_data = dataset[dataset[target_col] == 0]
        target_zero_sampled = target_zero_data.sample(frac=reduce_frac, random_state=42)
        filtered_data = pd.concat([target_one_data, target_zero_sampled])
    
    return filtered_data

In [ ]:
def train_catboost_model(dataset):
    # Фильтруем данные для обучения на основе дат
    train_data = dataset

    # Разделяем признаки и целевую переменную
    X_train = train_data.drop(columns=['user_id', 'target', 'event_date'])  # Исключаем идентификаторы и target
    y_train = train_data['target']
    
    # Подготовка Pool объектов для CatBoost с учетом категориальных фичей
    cat_features = ['day_of_week']  # Укажите категориальные признаки
    train_pool = Pool(data=X_train, label=y_train, cat_features=cat_features)
    
    # Учет дисбаланса классов с использованием scale_pos_weight
    # Используем соотношение классов в тренировочном наборе
    scale_pos_weight = len(y_train[y_train == 0]) / max(1, len(y_train[y_train == 1]))

    # Инициализация и обучение модели CatBoost
    model = CatBoostClassifier(
        iterations=300,
        learning_rate=0.1,
        depth=6,
        eval_metric='AUC',  # Метрика на обучении
        scale_pos_weight=scale_pos_weight,
        early_stopping_rounds=50,   # Ранняя остановка на обучении
        use_best_model=False,       # Отключаем выбор лучшей модели на валидации
        random_seed=42,
        verbose=100
        )
    
    # Обучение модели на всех данных
    model.fit(train_pool)
    
    return model

In [ ]:
# Загружаем данные train для блока 1
train_block_1 = pd.read_parquet(train_path_1)
test_block_1 = pd.read_parquet(test_path_1)

# Применяем фильтрацию к обучающим данным
train_block_1 = filter_data_for_training(train_block_1, test_block_1, 'train_block_1')

# Шаг 1: Удалите ссылку на DataFrame
del test_block_1

# Шаг 2: Принудительно вызовите сборщик мусора
gc.collect()

# Определяем столбцы, которые будут использоваться в модели для train и test данных
columns_to_keep_train = [
    'user_id',  'target', 'day_of_week', 'goal_cost', 
    'goal_budget_day', 'event_date', 'user_ad_count', 
    'cumulative_shows_user_campaign_logcat', 'click_rate_adv_logcat', 
    'logcat_campaign_click_rate', 'days_since_campaign_start', 
    'daily_ad_count', 'days_since_last_interaction_adv_logcat'
]



# Фильтруем данные, оставляя только нужные столбцы
train_block_1 = train_block_1[columns_to_keep_train]

# Обучаем модель на train_block_1
model_1 = train_catboost_model(train_block_1)

# Сохраняем модель в указанный путь
model_1.save_model(catboost_model_path_1)

# Шаг 1: Удалите ссылку на DataFrame
del train_block_1

# Шаг 2: Принудительно вызовите сборщик мусора
gc.collect()

In [ ]:
# Загружаем данные train для блока 2
train_block_2 = pd.read_parquet(train_path_2)
test_block_2 = pd.read_parquet(test_path_2)

train_block_2 = filter_data_for_training(train_block_2, test_block_2, 'train_block_2')

# Шаг 1: Удалите ссылку на DataFrame
del test_block_2

# Шаг 2: Принудительно вызовите сборщик мусора
gc.collect()


# Определяем столбцы, которые будут использоваться в модели для train и test данных
columns_to_keep_train = [
    'user_id', 'event_date', 'target', 'day_of_week', 
    'goal_budget_day', 'user_ad_count', 'cumulative_shows_logcat', 
    'click_rate_logcat', 'logcat_click_rate', 'days_since_campaign_start', 
    'daily_ad_count_logcat', 'days_since_last_interaction_logcat'
]



# Фильтруем данные, оставляя только нужные столбцы
train_block_2 = train_block_2[columns_to_keep_train]

# Обучаем модель на train_block_2
model_2 = train_catboost_model(train_block_2)

# Сохраняем модель в указанный путь
model_2.save_model(catboost_model_path_2)

# Шаг 1: Удалите ссылку на DataFrame
del train_block_2

# Шаг 2: Принудительно вызовите сборщик мусора
gc.collect()

In [ ]:
# Загружаем данные train для блока 3
train_block_3 = pd.read_parquet(train_path_3)
test_block_3 = pd.read_parquet(test_path_3)

train_block_3 = filter_data_for_training(train_block_3, test_block_3, 'train_block_3')

# Шаг 1: Удалите ссылку на DataFrame
del test_block_3

# Шаг 2: Принудительно вызовите сборщик мусора
gc.collect()

# Определяем необходимые столбцы для train и test данных
columns_to_keep_train = [
    'user_id', 'event_date', 'target', 'day_of_week', 
    'goal_budget_day', 'user_ad_count', 'click_rate', 'logcat_click_rate', 
    'days_since_campaign_start'
]



# Оставляем только нужные столбцы в train_block_3 и test_block_3
train_block_3 = train_block_3[columns_to_keep_train]

# Обучаем модель на train_block_3
model_3 = train_catboost_model(train_block_3)

# Сохраняем модель в указанный путь
model_3.save_model(catboost_model_path_3)

# Шаг 1: Удалите ссылку на DataFrame
del train_block_3

# Шаг 2: Принудительно вызовите сборщик мусора
gc.collect()

In [ ]:
model_1 = CatBoostClassifier()
model_1.load_model(catboost_model_path_1)
model_2 = CatBoostClassifier()
model_2.load_model(catboost_model_path_2)
model_3 = CatBoostClassifier()
model_3.load_model(catboost_model_path_3)



def predict_with_model(model, test_data, user_id_col='user_id', adv_campaign_id_col='adv_campaign_id'):
    # Определяем категориальные фичи
    cat_features = ['day_of_week']
    
    # Создаем Pool для тестовых данных с учетом cat_features
    test_pool = Pool(data=test_data.drop(columns=[user_id_col, adv_campaign_id_col]), cat_features=cat_features)
    
    # Выполняем предсказание вероятности для положительного класса
    test_data['predict'] = model.predict_proba(test_pool)[:, 1]
    
    # Возвращаем только необходимые столбцы
    return test_data[[user_id_col, adv_campaign_id_col, 'predict']]



In [ ]:
# Загрузка и выбор нужных столбцов для каждого блока тестовых данных
columns_to_keep_test_1 = [
    'user_id','adv_campaign_id', 'day_of_week', 'goal_cost', 
    'goal_budget_day', 'user_ad_count', 'cumulative_shows_user_campaign_logcat', 
    'click_rate_adv_logcat', 'logcat_campaign_click_rate', 
    'days_since_campaign_start', 'daily_ad_count', 
    'days_since_last_interaction_adv_logcat'
]

test_block_1 = pd.read_parquet(test_path_1)
test_block_1 = test_block_1[columns_to_keep_test_1]

columns_to_keep_test_2 = [
    'user_id', 'adv_campaign_id', 'day_of_week', 'goal_budget_day', 
    'user_ad_count', 'cumulative_shows_logcat', 'click_rate_logcat', 
    'logcat_click_rate', 'days_since_campaign_start', 'daily_ad_count_logcat', 
    'days_since_last_interaction_logcat'
]

test_block_2 = pd.read_parquet(test_path_2)
test_block_2 = test_block_2[columns_to_keep_test_2]

columns_to_keep_test_3 = [
    'user_id','adv_campaign_id', 'day_of_week', 'goal_budget_day', 
    'days_since_campaign_start', 'user_ad_count', 'click_rate', 'logcat_click_rate'
]

test_block_3 = pd.read_parquet(test_path_3)
test_block_3 = test_block_3[columns_to_keep_test_3]

# Убедитесь, что day_of_week является категориальной фичей
test_block_1['day_of_week'] = test_block_1['day_of_week'].astype('category')
test_block_2['day_of_week'] = test_block_2['day_of_week'].astype('category')
test_block_3['day_of_week'] = test_block_3['day_of_week'].astype('category')

# Выполняем предсказание после преобразования типов
# Получаем предсказания для каждого блока и сохраняем их в соответствующие файлы
result_1 = predict_with_model(model_1, test_block_1)
result_1.to_csv(output_file_path_1, index=False)  # Сохраняем результат для блока 1

result_2 = predict_with_model(model_2, test_block_2)
result_2.to_csv(output_file_path_2, index=False)  # Исправлено: сохраняем результат для блока 2

result_3 = predict_with_model(model_3, test_block_3)
result_3.to_csv(output_file_path_3, index=False)  # Исправлено: сохраняем результат для блока 3

# Объединяем предсказания из всех блоков в один DataFrame
submission = pd.concat([result_1, result_2, result_3], ignore_index=True)

# Сохраняем объединённый результат в файл submission.csv
submission.to_csv('submission.csv', index=False)
print("Файл submission.csv успешно сохранен.")

In [ ]:
def analyze_feature_importance(model, X_train, model_name):
    """
    Выводит важность признаков и строит графики SHAP для указанной модели.
    
    Параметры:
    - model: обученная модель CatBoostClassifier
    - X_train: тренировочный набор данных (без столбцов user_id, target, event_date)
    - model_name: имя модели (для отображения в графиках)
    """
    # Получаем важность признаков из CatBoost
    feature_importances = model.get_feature_importance(type="FeatureImportance")
    feature_names = X_train.columns
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': feature_importances
    }).sort_values(by="Importance", ascending=False)
    
    # Выводим таблицу важности признаков
    print(f"\nВажность признаков для {model_name}:")
    print(importance_df)
    
    # Строим график важности признаков
    plt.figure(figsize=(10, 6))
    importance_df.plot(kind='barh', x='Feature', y='Importance', legend=False, title=f'Feature Importance for {model_name}')
    plt.gca().invert_yaxis()
    plt.show()
    
    # Оценка важности признаков с помощью SHAP
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_train)
    
    print(f"\nSHAP Summary Plot for {model_name}:")
    shap.summary_plot(shap_values, X_train, plot_type="bar", show=False)
    plt.title(f"SHAP Feature Importance for {model_name}")
    plt.show()
    
    #print(f"\nSHAP Beeswarm Plot for {model_name}:")
    #shap.summary_plot(shap_values, X_train, show=False)
    #plt.title(f"SHAP Feature Influence for {model_name}")
    #plt.show()

# Загружаем данные train для блока 1
train_block_1 = pd.read_parquet(train_path_1)
test_block_1 = pd.read_parquet(test_path_1)
train_block_1 = filter_data_for_training(train_block_1, test_block_1, 'train_block_1')
del test_block_1
gc.collect()
columns_to_keep_train_1 = [
    'user_id', 'target', 'day_of_week', 'goal_cost', 
    'goal_budget_day', 'event_date', 'user_ad_count', 
    'cumulative_shows_user_campaign_logcat', 'click_rate_adv_logcat', 
    'logcat_campaign_click_rate', 'days_since_campaign_start', 
    'daily_ad_count', 'days_since_last_interaction_adv_logcat'
]
train_block_1 = train_block_1[columns_to_keep_train_1]

train_block_2 = pd.read_parquet(train_path_2)
test_block_2 = pd.read_parquet(test_path_2)
train_block_2 = filter_data_for_training(train_block_2, test_block_2, 'train_block_2')
del test_block_2
gc.collect()
columns_to_keep_train_2 = [
    'user_id',  'event_date', 'target', 'day_of_week', 
    'goal_budget_day', 'user_ad_count', 'cumulative_shows_logcat', 
    'click_rate_logcat', 'logcat_click_rate', 'days_since_campaign_start', 
    'daily_ad_count_logcat', 'days_since_last_interaction_logcat'
]
train_block_2 = train_block_2[columns_to_keep_train_2]

train_block_3 = pd.read_parquet(train_path_3)
test_block_3 = pd.read_parquet(test_path_3)
train_block_3 = filter_data_for_training(train_block_3, test_block_3, 'train_block_3')
del test_block_3
gc.collect()
columns_to_keep_train_3 = [
    'user_id', 'event_date', 'target', 'day_of_week', 
    'goal_budget_day', 'user_ad_count', 'click_rate', 'logcat_click_rate', 
    'days_since_campaign_start'
]
train_block_3 = train_block_3[columns_to_keep_train_3]

##



In [ ]:
# Вызываем функцию для каждой модели и соответствующего тренировочного набора
print("Анализ важности признаков для model_1:")
analyze_feature_importance(model_1, train_block_1.drop(columns=['user_id', 'target', 'event_date']), "Model 1")

print("Анализ важности признаков для model_2:")
analyze_feature_importance(model_2, train_block_2.drop(columns=['user_id', 'target', 'event_date']), "Model 2")

print("Анализ важности признаков для model_3:")
analyze_feature_importance(model_3, train_block_3.drop(columns=['user_id', 'target', 'event_date']), "Model 3")